# 尝试使用cosine similarity

milvus 默认使用的是ip，就是向量内积？

因此如果需要使用余弦相似度，需要进行设置？

In [1]:
%%time

import logging
import sys

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding

from llama_index.llms.openai_like import OpenAILike

from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore

CPU times: user 3.55 s, sys: 432 ms, total: 3.98 s
Wall time: 3.65 s


In [2]:
%%time

# llm
llm = OpenAILike(model="qwen14b", 
                 api_base="http://192.168.0.72:3000/v1", 
                 api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
                 is_chat_model=True,
                 temperature=0.1
                )

CPU times: user 108 µs, sys: 14 µs, total: 122 µs
Wall time: 124 µs


In [3]:
%%time

# embedding
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="dztech/bge-large-zh:v1.5",
    # model_name="bge-m3:latest",
    base_url="http://192.168.0.72:11435",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

CPU times: user 67 µs, sys: 9 µs, total: 76 µs
Wall time: 78.4 µs


In [4]:
Settings.chunk_size=128
Settings.chunk_overlap=10

Settings.llm =llm
Settings.embed_model = ollama_embedding

Settings

_Settings(_llm=OpenAILike(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f64a3df4dc0>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x7f64aeba00d0>, completion_to_prompt=<function default_completion_to_prompt at 0x7f64aebd28c0>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, model='qwen14b', temperature=0.1, max_tokens=None, logprobs=None, top_logprobs=0, additional_kwargs={}, max_retries=3, timeout=60.0, default_headers=None, reuse_client=True, api_key='sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750', api_base='http://192.168.0.72:3000/v1', api_version='', context_window=3900, is_chat_model=True, is_function_calling_model=False, tokenizer=None), _embed_model=OllamaEmbedding(model_name='dztech/bge-large-zh:v1.5', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f64a3df4dc0>, num_workers=None, base_url='http://1

In [5]:
%%time

from milvus import default_server
from pymilvus import connections, utility

default_server.set_base_dir('milvus_data')

# (OPTIONAL) if you want cleanup previous data
default_server.cleanup()

# Start your milvus server
default_server.start()

# Now you could connect with localhost and the given port
# Port is defined by default_server.listen_port
connections.connect(host='127.0.0.1', port=default_server.listen_port)

# Check if the server is ready.
print(utility.get_server_version())

v2.3.5-lite
CPU times: user 3.17 s, sys: 263 ms, total: 3.43 s
Wall time: 6.44 s


In [6]:
%%time

# load documents
documents = SimpleDirectoryReader(input_files=["./books1/故乡.txt"]).load_data()
print("Document ID:", documents[0].doc_id)

Document ID: 29ff2226-2529-4520-b11e-bae1ffb00ad5
CPU times: user 9.6 ms, sys: 0 ns, total: 9.6 ms
Wall time: 8.91 ms


In [7]:
%%time

vector_store = MilvusVectorStore(
    dim=1024, 
    overwrite=True,
    similarity_metric="COSINE",
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

CPU times: user 404 ms, sys: 14 ms, total: 418 ms
Wall time: 13.1 s


In [18]:
%%time

# Query Data
query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=3
)

CPU times: user 161 µs, sys: 0 ns, total: 161 µs
Wall time: 163 µs


In [19]:
%%time

streaming_response = query_engine.query("闰土是谁")
streaming_response.print_response_stream()
print()

闰土是小说《故乡》中的人物，他是主人公鲁迅（即文中的“我”）儿时的朋友，和“我”同年出生，因为生于闰月，五行缺土，所以他的父亲给他取名为闰土。他是一个擅长捕鸟的孩子。



CPU times: user 88.2 ms, sys: 12.9 ms, total: 101 ms
Wall time: 6.77 s


In [20]:
for node in streaming_response.source_nodes:
    print(node.score,node.metadata["file_name"],(lambda s: s.replace('\n', '')[:10] + '..' if len(s.replace('\n', '')) > 10 else s.replace('\n', ''))(node.text))

0.6530978679656982 故乡.txt 我这时很兴奋，但不知..
0.6412548422813416 故乡.txt 闰土来管祭器的。　　..
0.6376977562904358 故乡.txt 一日是天气很冷的午后..


In [25]:
%%time

from llama_index.core.postprocessor import SimilarityPostprocessor

postprocessor = SimilarityPostprocessor(similarity_cutoff=0.6)

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    node_postprocessors=[postprocessor]
)

streaming_response = query_engine.query("闰土是谁")
streaming_response.print_response_stream()
print()

闰土是一个小说中的人物，他是鲁迅先生的作品《故乡》中的一个重要角色。他是一个农村少年，与主角鲁迅（即文中的“我”）有着童年时的友谊。闰土的名字来源于他出生的月份是闰月，并且五行缺土。他在故事中代表了农村生活和那个时代的一种典型形象。


CPU times: user 131 ms, sys: 3.26 ms, total: 135 ms
Wall time: 5.54 s


In [26]:
for node in streaming_response.source_nodes:
    print(node.score,node.metadata["file_name"],(lambda s: s.replace('\n', '')[:20] + '..' if len(s.replace('\n', '')) > 10 else s.replace('\n', ''))(node.text))

0.6530978679656982 故乡.txt 我这时很兴奋，但不知道怎么说才好，只是说..
0.6412548422813416 故乡.txt 闰土来管祭器的。　　我的父亲允许了；我也..
0.6376977562904358 故乡.txt 一日是天气很冷的午后，我吃过午饭，坐着喝..
0.6281930804252625 故乡.txt “阿呀，老太太真是……这成什么规矩。那时..
0.6249887347221375 故乡.txt 工的叫长工；按日给人做工的叫短工；自己也..
0.6214910745620728 故乡.txt “我们沙地里，潮汛要来的时候，就有许多跳..
0.619132399559021 故乡.txt 他见人很怕羞，只是不怕我，没有旁人的时候..
0.6188885569572449 故乡.txt 我于是日日盼望新年，新年到，闰土也就到了..
0.6178779006004333 故乡.txt “老太太。信是早收到了。我实在喜欢的不得..
0.6165617108345032 故乡.txt ，反从他的胯下逃走了。　　这少年便是闰土..
0.6116492748260498 故乡.txt ”　　“回来？你怎么还没有走就想回来了。..
0.6069682836532593 故乡.txt ”　　⑶大祭祀的值年：封建社会中的大家族..
0.6067788600921631 故乡.txt 母亲叫闰土坐，他迟疑了一回，终于就了坐，..
0.6026837229728699 故乡.txt 我希望他们不再像我，又大家隔膜起来……然..
0.6015970706939697 故乡.txt 虽然我一见便知道是闰土，但又不是我这记忆..


In [27]:
streaming_response = query_engine.query("孔乙己是谁")
streaming_response.print_response_stream()
print()

Empty Response


In [28]:
for node in streaming_response.source_nodes:
    print(node.score,node.metadata["file_name"],(lambda s: s.replace('\n', '')[:20] + '..' if len(s.replace('\n', '')) > 10 else s.replace('\n', ''))(node.text))

In [30]:
query_engine.get_prompts().keys()

dict_keys(['response_synthesizer:text_qa_template', 'response_synthesizer:refine_template'])

In [56]:
%%time

from llama_index.core import PromptTemplate

# shakespeare!
qa_prompt_tmpl_str = (
    "上下文信息见下面\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "根据给定上下文信息而不是先验知识， "
    "回答下面问题。\n"
    "如果没有上下文信息，则回答未找到相关信息。\n"
    "问题: 上下文中是否有，{query_str}，如果未找到就回答不知道\n"
    "回答: "
)
qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

CPU times: user 103 µs, sys: 0 ns, total: 103 µs
Wall time: 106 µs


In [38]:
%%time

streaming_response = query_engine.query("孔乙己是谁")
streaming_response.print_response_stream()
print()

Empty Response
CPU times: user 21.8 ms, sys: 146 µs, total: 21.9 ms
Wall time: 377 ms


In [41]:
%%time

streaming_response = query_engine.query("宏儿是谁")
streaming_response.print_response_stream()
print()

Empty Response
CPU times: user 14.2 ms, sys: 7.94 ms, total: 22.2 ms
Wall time: 413 ms


In [40]:
%%time

streaming_response = query_engine.query("闰土是谁")
streaming_response.print_response_stream()
print()

闰土是小说《故乡》中的人物，他是鲁迅笔下的一个少年朋友，与主角鲁迅有着深厚的童年记忆。













CPU times: user 88 ms, sys: 2.47 ms, total: 90.4 ms
Wall time: 6.75 s


In [42]:
%%time

streaming_response = query_engine.query("水生是谁")
streaming_response.print_response_stream()
print()

Empty Response
CPU times: user 21.5 ms, sys: 1.25 ms, total: 22.8 ms
Wall time: 294 ms


In [43]:
%%time

from llama_index.core.postprocessor import SimilarityPostprocessor

postprocessor = SimilarityPostprocessor(similarity_cutoff=0.5)

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    node_postprocessors=[postprocessor]
)

streaming_response = query_engine.query("闰土是谁")
streaming_response.print_response_stream()
print()

闰土是《故乡》中描述的少年形象，他的存在象征着作者对于过去美好时光的回忆。
CPU times: user 173 ms, sys: 6.94 ms, total: 180 ms
Wall time: 12.3 s


In [59]:
%%time

# from llama_index.core.postprocessor import SimilarityPostprocessor

postprocessor = SimilarityPostprocessor(similarity_cutoff=0.5)

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    node_postprocessors=[postprocessor]
)

streaming_response = query_engine.query("水生是谁")
streaming_response.print_response_stream()
print()

Empty Response
CPU times: user 22.5 ms, sys: 202 µs, total: 22.7 ms
Wall time: 275 ms


In [46]:
streaming_response = query_engine.query("宏儿是谁")
streaming_response.print_response_stream()
print()

闰土的孩子。


In [61]:
%%time

from llama_index.core.postprocessor import SimilarityPostprocessor

postprocessor = SimilarityPostprocessor(similarity_cutoff=0.5)

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    node_postprocessors=[postprocessor]
)

streaming_response = query_engine.query("孔乙己是谁")
streaming_response.print_response_stream()
print()

Empty Response
CPU times: user 19.5 ms, sys: 4.39 ms, total: 23.9 ms
Wall time: 303 ms
